# 3D segmentation using CellPose

This notebook demonstrates a complete 3D deep learning segmentation workflow using CellPose. It loads a 3D volume with BioIO and Dask for memory efficiency, visualizes it in Napari's interactive viewer, applies GPU-accelerated Cellpose for automated 3D cell segmentation, and overlays the resulting masks on the original volume for evaluation. It requires bioio, napari, and cellpose with GPU support.

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Load and visualize 3D microscopy data** using BioIO and Napari
    - Understand how to efficiently handle large 3D images with Dask arrays
    - Navigate and render 3D volumes in Napari's viewer

2. **Apply deep learning segmentation to 3D data** using Cellpose
    - Configure Cellpose for 3D segmentation tasks
    - Understand key parameters: `diameter`, `z_axis`, and `do_3D`
    - Leverage GPU acceleration for faster processing

3. **Evaluate and refine segmentation results**
    - Visualize segmentation masks overlaid on original data
    - Identify when parameters need adjustment (e.g., cell diameter)
    - Recognize scenarios requiring model retraining

4. **Develop practical skills for 3D image analysis workflows**
    - Build reproducible analysis pipelines in Jupyter notebooks
    - Work with real biological imaging data (Lund dataset)
    - Understand the relationship between image properties and segmentation parameters

## Load the data

Using [BioIO](https://bioio-devs.github.io/bioio/OVERVIEW.html) with [Dask](https://docs.dask.org/en/stable/) lets us stream the 3D volume lazily so it loads without blowing up memory before sending it to Napari and Cellpose.

In [ ]:
from bioio import BioImage

In [2]:
image_handle = BioImage("../data/Lund.tif")

## Visualize in Napari

We'll use [Napari](https://napari.org/stable/) to interactively visualize the 3D volume.

In [3]:
import napari

# Use Dask for lazy loading of the heavy 3D image
# BioImage's .dask_data provides a Dask array without loading into memory
image_data = image_handle.dask_data.squeeze()
image_data

dask.array<getitem, shape=(100, 256, 256), dtype=uint16, chunksize=(100, 256, 256), chunktype=numpy.ndarray>

In [4]:
# Create a Napari viewer and add the 3D image
viewer = napari.Viewer()
viewer.add_image(image_data, name='Lund volume')
napari.run()

## Load and run CellPose in GPU

Cellpose is an open-source deep learning model for general cell and nucleus segmentation in microscopy images. In this notebook, it will be used to segment nuclei in the 3D volume. For details, see the [Cellpose documentation](https://cellpose.readthedocs.io/).

In [5]:
from cellpose import io, models, core

# Initialize Cellpose model with GPU enabled
model = models.CellposeModel(gpu=True)

io.logger_setup() # run this to get printing of progress

#Check if colab notebook instance has GPU access
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

2026-02-20 20:17:09,194 [INFO] WRITING LOG OUTPUT TO C:\Users\corba\.cellpose\run.log
2026-02-20 20:17:09,195 [INFO] 
cellpose version: 	4.0.8 
platform:       	win32 
python version: 	3.11.14 
torch version:  	2.10.0
2026-02-20 20:17:09,196 [INFO] ** TORCH CUDA version installed and working. **


In [6]:
# Run Cellpose segmentation in 3D mode
masks, flows, styles = model.eval(
    image_data.compute(),
    diameter=100,  # Adjust based on typical cell size in pixels
    z_axis=0,  # Specify the z-axis for 3D data
    do_3D=True  # Enable 3D segmentation
)

2026-02-20 20:17:09,546 [INFO] running YX: 100 planes of size (76, 76)
2026-02-20 20:17:17,358 [INFO] 100%|##########| 13/13 [00:07<00:00,  1.67it/s]
2026-02-20 20:17:17,376 [INFO] running ZY: 76 planes of size (100, 76)
2026-02-20 20:17:21,276 [INFO] 100%|##########| 10/10 [00:03<00:00,  2.57it/s]
2026-02-20 20:17:21,291 [INFO] running ZX: 76 planes of size (100, 76)
2026-02-20 20:17:25,189 [INFO] 100%|##########| 10/10 [00:03<00:00,  2.57it/s]
2026-02-20 20:17:25,212 [INFO] network run in 15.67s
2026-02-20 20:17:26,753 [INFO] masks created in 0.87s


In [7]:
viewer = napari.Viewer()
viewer.add_image(image_data, name="Lund volume", rendering="attenuated_mip")
viewer.add_labels(masks, name="Cellpose masks", opacity=0.5)
viewer.dims.ndisplay = 3
napari.run()

2026-02-20 20:17:29,719 [INFO] No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'


## Questions

1. Is there any parameter that should be corrected when running CellPose?

2. Do we need to retrain the network?

### Harder questions

1. Can you run Cellpose on the dataset called `lund1051_resampled.tif`? What happens when you choose the right size of the cell nucleus?